In [ ]:
from petastorm import make_reader
import numpy as np
import sys
import os
import pandas as pd
import seaborn as sn
import mogptk as mo
sys.path.append(os.path.join(os.getcwd(), ".."))
path_formatted_glasgow_disease = "/workspaces/maitrise/data/20220902_data_physio_formatted_merged/merged/dataParquet"
path_petastorm_disease = f"file:///{path_formatted_glasgow_disease}"
path_formatted_glasgow = "/workspaces/maitrise/data/20221006_physio_quality/set-a/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"
path_csv_ref_label = "/workspaces/maitrise/data/20221006_physio_quality/set-a/REFERENCE.csv"

In [ ]:
##Dataset Organization : 
with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        if data.signal_quality == "acceptable".encode():
            break
        else : 
            continue

ECG_lead = [e.decode('UTF-8') for e in data.signal_names]
fs = data.sampling_frequency
dico_ECG = {}
for i,j in zip(ECG_lead,range(12)):
    
    dico_ECG[i] = data.signal[:,j]
N = len(dico_ECG[ECG_lead[0]])
print(data)

In [ ]:
##Dataset with MOGPTK
t = np.linspace(0,N/fs,N)

# dataset = mo.DataSet(mo.Data(t,dico_ECG[ECG_lead[0]],name = ECG_lead[0]),
#                      mo.Data(t,dico_ECG[ECG_lead[1]],name = ECG_lead[1]),
#                      mo.Data(t,dico_ECG[ECG_lead[2]],name = ECG_lead[2]),
#                      mo.Data(t,dico_ECG[ECG_lead[3]],name = ECG_lead[3]),
#                      mo.Data(t,dico_ECG[ECG_lead[4]],name = ECG_lead[4]),
#                      mo.Data(t,dico_ECG[ECG_lead[5]],name = ECG_lead[5]),
#                      mo.Data(t,dico_ECG[ECG_lead[6]],name = ECG_lead[6]),
#                      mo.Data(t,dico_ECG[ECG_lead[7]],name = ECG_lead[7]),
#                      mo.Data(t,dico_ECG[ECG_lead[8]],name = ECG_lead[8]),
#                      mo.Data(t,dico_ECG[ECG_lead[9]],name = ECG_lead[9]),
#                      mo.Data(t,dico_ECG[ECG_lead[10]],name = ECG_lead[10]),
#                      mo.Data(t,dico_ECG[ECG_lead[11]],name = ECG_lead[11]))

dataset = mo.DataSet(mo.Data(t,dico_ECG[ECG_lead[0]],name = ECG_lead[0]),
                     mo.Data(t,dico_ECG[ECG_lead[1]],name = ECG_lead[1]),
                     mo.Data(t,dico_ECG[ECG_lead[2]],name = ECG_lead[2]))


In [ ]:
##Removing datapoints in dataset :
class TransformWhiten(mo.TransformBase):
    """
    Transform the data so it has mean 0 and variance 1
    """
    def __init__(self):
        pass

    def set_data(self, y,x=None):
        # take only the non-removed observations
        self.mean = y.mean()
        self.std = y.std()

    def forward(self, y, x=None):
        return (y - self.mean) / self.std

    def backward(self, y, x=None):
        return (y * self.std) + self.mean


start_ = 1
end_ = 3
test = 1
for channel in dataset:
    channel.filter(start_, end_)

for i, channel in enumerate(dataset):
    if i == 0:
        channel.remove_range(end_-test, end_)

for data in dataset:
    data.remove_randomly(pct=0.5)

for channel in dataset:
    channel.transform(TransformWhiten())
    channel.transform(mo.TransformLog())

dataset.plot('Normalized Dataset')


In [ ]:
# create model
model = mo.MOSM(dataset, Q=4)
# initial estimation of parameters
model.init_parameters()
# train
model.train(iters=1000, verbose=True, plot=True)
# predict and plot
model.plot_prediction(title='Trained model')